In [ ]:
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cab
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gc
#from featexp import get_univariate_plots#用于特征筛选，需要先安装featexp
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif']=['Simhei']
plt.rcParams['axes.unicode_minus']=False
import json

In [ ]:
base_info=pd.read_csv('train/base_info.csv')#企业的基本信息
annual_report_info=pd.read_csv('train/annual_report_info.csv')#企业的年报基本信息
tax_info=pd.read_csv('train/tax_info.csv')#企业的纳税信息
change_info=pd.read_csv('train/tax_info.csv')#变更信息
news_info=pd.read_csv('train/news_info.csv')#舆情信息
other_info=pd.read_csv('train/other_info.csv')#其它信息
entprise_info=pd.read_csv('train/entprise_info.csv')#企业标注信息{0: 13884, 1: 981}
entprise_evaluate=pd.read_csv('entprise_evaluate.csv')#未标注信息

print('base_info shape:',base_info.shape,'id unique:',len(base_info['id'].unique()))
print('annual_report_info shape:',annual_report_info.shape,'id unique:',len(annual_report_info['id'].unique()))
print('tax_info shape:',tax_info.shape,'id unique:',len(tax_info['id'].unique()))
print('change_info shape:',change_info.shape,'id unique:',len(change_info['id'].unique()))
print('news_info shape:',news_info.shape,'id unique:',len(news_info['id'].unique()))
print('other_info shape:',other_info.shape,'id unique:',len(other_info['id'].unique()))
print('entprise_info shape:',entprise_info.shape,'id unique:',len(entprise_info['id'].unique()))
print('entprise_evaluate shape:',entprise_evaluate.shape,'id unique:',len(entprise_evaluate['id'].unique()))

In [ ]:
#下面看一下具有企业年报信息和纳税信息的企业有多少是非法集资的企业
#首先筛选出非法集资的企业
illegal_id_list=[]
legal_id_list=[]
for index,name_id,flag in entprise_info.itertuples():
    if flag==1:
        illegal_id_list.append(name_id)
    else:
        legal_id_list.append(name_id)
len(legal_id_list),len(illegal_id_list),len(legal_id_list)/len(illegal_id_list)

In [ ]:
#..................年报基本信息信息数据...................
cnt_list_annual={'-1':0,'0':0,'1':0}
for i in annual_report_info['id'].unique():
    if i in illegal_id_list:
        cnt_list_annual['1']+=1
    elif i in legal_id_list:
        cnt_list_annual['0']+=1
    else:
        cnt_list_annual['-1']+=1
#具有年报基本信息的企业中，有536违法；2800合法；5601为测试集
print("具有年报基本信息的企业中，有{}违法；{}合法；{}为测试集".format(cnt_list_annual['1'],cnt_list_annual['0'],cnt_list_annual['-1']))
#合法/违法:5.223880597014926，说明具有年报信息的企业，是非法的概率很高 由此可见，年报信息很重要，这是十分重要的特征
print("具有年报基本信息的企业中：合法/违法:{}".format(cnt_list_annual['0']/cnt_list_annual['1']))
print("不具有年报基本信息的企业中：合法/违法:{}".format((len(legal_id_list)-cnt_list_annual['0'])/(len(illegal_id_list)-cnt_list_annual['1'])))
#由此可见，纳税信息很重要，这是十分重要的特征
#...........................纳税信息news_info....................
cnt_list_annual={'-1':0,'0':0,'1':0}
for i in tax_info['id'].unique():
    if i in illegal_id_list:
        cnt_list_annual['1']+=1
    elif i in legal_id_list:
        cnt_list_annual['0']+=1
    else:
        cnt_list_annual['-1']+=1
#具有年报基本信息的企业中，有536违法；2800合法；5601为测试集
print("具有纳税基本信息的企业中，有{}违法；{}合法；{}为测试集".format(cnt_list_annual['1'],cnt_list_annual['0'],cnt_list_annual['-1']))
#合法/违法:5.223880597014926，说明具有年报信息的企业，是非法的概率很高 由此可见，年报信息很重要，这是十分重要的特征
print("具有纳税信息的企业中：合法/违法:{}".format(cnt_list_annual['0']/cnt_list_annual['1']))
print("不具纳税信息的企业中：合法/违法:{}".format((len(legal_id_list)-cnt_list_annual['0'])/(len(illegal_id_list)-cnt_list_annual['1'])))

In [ ]:
#处理base_info的数据
#空值大于0.5的列都删除掉
annual_report_info_clean=annual_report_info.dropna(thresh=annual_report_info.shape[0]*0.5,how='all',axis=1)
#对object类型进行编码
annual_report_info_clean['BUSSTNAME']=annual_report_info_clean['BUSSTNAME'].fillna("无")
dic = {'无':-1,'开业':0, '歇业':1, '停业':2, '清算':3}
buf = pd.DataFrame()
buf_group = annual_report_info_clean.groupby('BUSSTNAME',sort=False)
for name,group in buf_group:
    group['BUSSTNAME'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
buf=pd.DataFrame(buf_group).reset_index()
annual_report_info_clean=buf.drop(['ANCHEYEAR'],axis=1)
annual_report_info_clean

In [ ]:
#处理tax数据
tax_info_clean=tax_info.drop(['START_DATE','END_DATE'],axis=1)
tax_info_clean['TAX_CATEGORIES']=tax_info_clean['TAX_CATEGORIES'].fillna("无")
tax_info_clean['TAX_ITEMS']=tax_info_clean['TAX_ITEMS'].fillna("无")
#对object类型进行编码
# tax_info_clean['BUSSTNAME']=tax_infoclean['BUSSTNAME'].fillna("无")
dic={}
cate=tax_info.TAX_CATEGORIES.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf = pd.DataFrame()
buf_group = tax_info_clean.groupby('TAX_CATEGORIES',sort=False)
for name,group in buf_group:
    group['TAX_CATEGORIES'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)

#
dic={}
cate=buf.TAX_ITEMS.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('TAX_ITEMS',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['TAX_ITEMS'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
tax_info_clean=pd.DataFrame(buf_group).reset_index()
tax_info_clean

In [ ]:
# #处理base_info数据
base_info_clean=base_info.drop(['opscope','opfrom','opto'],axis=1)

#............................对object类型进行编码...............................
base_info_clean['industryphy']=base_info_clean['industryphy'].fillna("无")
base_info_clean['dom']=base_info_clean['dom'].fillna("无")
base_info_clean['opform']=base_info_clean['opform'].fillna("无")
base_info_clean['oploc']=base_info_clean['oploc'].fillna("无")
#
dic={}
cate=base_info_clean.industryphy.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf = pd.DataFrame()
buf_group = base_info_clean.groupby('industryphy',sort=False)
for name,group in buf_group:
    group['industryphy'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 1....')
#
dic={}
cate=buf.dom.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('dom',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['dom'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 2....')
#
dic={}
cate=buf.opform.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('opform',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['opform'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 3....')
#
dic={}
cate=buf.oploc.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('oploc',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['oploc'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 4....')
#
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
base_info_clean=pd.DataFrame(buf_group).reset_index()
#
print('编码完毕.................')

In [ ]:
base_info_clean.to_csv('base_info_clean.csv',index=False)

In [ ]:
base_info_clean = pd.read_csv('base_info_clean.csv')

In [ ]:
def bucket(name,bucket_len):
    gap_list=[base_info_clean[name].quantile(i/bucket_len) for i in range(bucket_len+1)]
    len_data=len(base_info_clean[name])
    new_col=[]
    for i in base_info_clean[name].values:
        for j in range(len(gap_list)):
            if gap_list[j]>=i:
                encode=j
                break
        new_col.append(encode)
    return new_col
#注册资本_实缴资本
base_info_clean['regcap_reccap']=base_info_clean['regcap']-base_info_clean['reccap']
#注册资本分桶
base_info_clean['regcap']=base_info_clean['regcap'].fillna(base_info_clean['regcap'].median())
base_info_clean['bucket_regcap']=bucket('regcap',5)
#实缴资本分桶
base_info_clean['reccap']=base_info_clean['reccap'].fillna(base_info_clean['reccap'].median())
base_info_clean['bucket_reccap']=bucket('reccap',5)
#注册资本_实缴资本分桶
base_info_clean['regcap_reccap']=base_info_clean['regcap_reccap'].fillna(base_info_clean['regcap_reccap'].median())
base_info_clean['bucket_regcap_reccap']=bucket('regcap_reccap',5)
print('分桶完毕.................')

In [ ]:
def cross_two(name_1,name_2):
    new_col=[]
    encode=0
    dic={}
    val_1=base_info[name_1]
    val_2=base_info[name_2]
    for i in tqdm(range(len(val_1))):
        tmp=str(val_1[i])+'_'+str(val_2[i])
        if tmp in dic:
            new_col.append(dic[tmp])
        else:
            dic[tmp]=encode
            new_col.append(encode)
            encode+=1
    return new_col
#作企业类型-小类的交叉特征
base_info_clean['enttypegb']=base_info_clean['enttypegb'].fillna("无")
base_info_clean['enttypeitem']=base_info_clean['enttypeitem'].fillna("无")
new_col=cross_two('enttypegb','enttypeitem')#作企业类型-小类的交叉特征
base_info_clean['enttypegb_enttypeitem']=new_col
#
#行业类别-细类的交叉特征
base_info_clean['industryphy']=base_info_clean['industryphy'].fillna("无")
base_info_clean['industryco']=base_info_clean['industryco'].fillna("无")
new_col=cross_two('industryphy','industryco')#作企业类型-小类的交叉特征
base_info_clean['industryphy_industryco']=new_col
print('交叉特征完毕.................')

In [ ]:
cat_features=['industryphy','dom','opform','oploc','bucket_regcap',
              'bucket_reccap','bucket_regcap_reccap',
              'enttypegb','enttypeitem','enttypegb_enttypeitem',
              'industryphy','industryco','industryphy_industryco',
              'adbusign','townsign','regtype','TAX_CATEGORIES'
             ]

In [ ]:
#暂时可以利用企业基本信息，企业纳税信息，企业年度财报信息做义工merge然后进行我们的分类工作
all_data=base_info_clean.merge(annual_report_info_clean,how='outer')
all_data=all_data.merge(tax_info_clean,how='outer')
all_data=all_data.fillna(-1)
all_data.shape,base_info.shape,annual_report_info.shape,tax_info.shape

In [ ]:
all_data[cat_features]=all_data[cat_features].astype(int)

In [ ]:

#train_data=all_data[all_data['id'].isin(entprise_info['id'].unique().tolist())]
#train_data=train_data.reset_index(drop=True)
train_df=all_data.merge(entprise_info)
train_data=train_df.drop(['id','label'],axis=1)
kind=train_df['label']
test_df=all_data[all_data['id'].isin(entprise_evaluate['id'].unique().tolist())]
test_df=test_df.reset_index(drop=True)
test_data=test_df.drop(['id'],axis=1)
train_data.shape,test_data.shape

In [ ]:
def eval_score(y_test,y_pre):
    _,_,f_class,_=precision_recall_fscore_support(y_true=y_test,y_pred=y_pre,labels=[0,1],average=None)
    fper_class={'合法':f_class[0],'违法':f_class[1],'f1':f1_score(y_test,y_pre)}
    return fper_class
#
def k_fold_serachParmaters(model,train_val_data,train_val_kind):
    mean_f1=0
    mean_f1Train=0
    n_splits=5
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
    for train, test in sk.split(train_val_data, train_val_kind):
        x_train = train_val_data.iloc[train]
        y_train = train_val_kind.iloc[train]
        x_test = train_val_data.iloc[test]
        y_test = train_val_kind.iloc[test]

        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        fper_class =  eval_score(y_test,pred)
        mean_f1+=fper_class['f1']/n_splits
        #print(fper_class)
        
        pred_Train = model.predict(x_train)
        fper_class_train =  eval_score(y_train,pred_Train)
        mean_f1Train+=fper_class_train['f1']/n_splits
    #print('mean valf1:',mean_f1)
    #print('mean trainf1:',mean_f1Train)
    return mean_f1

In [ ]:
n_estimators_list = []
for i in range(1,101):
    n_estimators_list.append(int(i))
min_samples_split_list = []
for i in range(2,30):
    min_samples_split_list.append(i)
max_depth_list = []
for i in range(2,30):
    max_depth_list.append(i)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve, GridSearchCV

def search_param(n_estimators,max_depth,min_samples_split):
    rf = RandomForestClassifier(oob_score=True, random_state=2020,
                    n_estimators= n_estimators,max_depth=max_depth,min_samples_split=min_samples_split)
    mean_f1=k_fold_serachParmaters(rf,train_data,kind)
    return mean_f1

#搜索最佳参数
param=[]
best=0
for n_estimators in n_estimators_list:
    print('n_estimators:',n_estimators)
    for min_samples_split in min_samples_split_list:
        for max_depth in max_depth_list:
            mean_f1=search_param(n_estimators,max_depth,min_samples_split)
            if mean_f1>best:
                param=[n_estimators,min_samples_split,max_depth]
                best=mean_f1
                print(param,best)